##### Copyright 2025 The AI Edge Torch Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Exporting Gemma3 270M with AI Edge Torch

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google-ai-edge/ai-edge-torch/blob/main/ai_edge_torch/generative/colabs/Gemma3_270M_convertion.ipynb)

In this colab, we will show you how to export a Gemma-3-270M model to LiteRT-LM format with AI Edge Torch.

It works with the base Gemma-3-270M-it model and its fine-tuned models. For later, checkout the [Full Model Fine-Tune using Hugging Face Transformers](https://ai.google.dev/gemma/docs/core/huggingface_text_full_finetune) tutorial.

#Prerequisite for exporting google/gemma-3-270m-it

- Create HuggingFace token with permission access to
  - google/gemma-3-270m-it

  This is needed to download the checkpoint and tokenizer.

- Open Colab Secrets: In your Google Colab notebook, locate the Secrets icon in the left-hand sidebar and click on it.
- Add a new secret: Click the "Add Secret" button.
- Name your secret: Enter "HF_TOKEN" for your token in the "Name" field.
- Paste your token: In the "Value" field, paste the actual token you want to store.

#Prerequisite for exporting fine-tuned model

- Access to the finetuned repo in Hugging Face Hub, or

- Access to the finetuned checkpoint


## Note: When running notebooks in this repository with Google Colab, some users may see the following warning message:

![Colab warning](https://github.com/google-ai-edge/ai-edge-torch/blob/main/docs/data/colab_warning.jpg?raw=true)

Please click `Restart Session` and run again.


This colab works with a free tier colab runtime.


In [ ]:
# @title Install dependencies and environment setup

!pip install ai-edge-litert-nightly==2.0.2.dev20250917
!pip uninstall -y tensorflow
!pip install ai-edge-torch-nightly==0.7.0.dev20250920  --force-reinstall

In [ ]:
# Setup Hugging Face Hub credentials

import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [ ]:
# @title Import needed packages.
from huggingface_hub import snapshot_download
from ai_edge_torch.generative.examples.gemma3 import gemma3
from ai_edge_torch.generative.utilities import converter
from ai_edge_torch.generative.utilities.export_config import ExportConfig
from ai_edge_torch.generative.layers import kv_cache

# Exporting the checkpoint to LiteRT-LM format.

In this example, we directly use the google/gemma-3-270m-it repo. But you can also replace it with your fine-tuned model directory or repo ID.

If you are following the fine-tune colab and storing your checkpoint to Google Drive as the default setup, you can point to the checkpoint with the followings instead of downloading the base checkpoint.

```
from google.colab import drive
drive.mount('/content/drive')
checkpoint_dir = '/content/drive/MyDrive/MyGemmaNPC'
```


In [ ]:
# @title Download checkpoint

checkpoint_dir = snapshot_download('google/gemma-3-270m-it')


# Convert to LiteRT-LM format

After the following cell, you will be able to download the exported `.litertlm` file under `/contents/`, which will be accessible from the `Files` pannel.

In [ ]:
output_dir = '/content/'

# Import the weights and build the PyTorch model
pytorch_model = gemma3.build_model_270m(checkpoint_dir)

# Setup the export configurations and parameters for text generation models.
export_config = ExportConfig()
export_config.kvcache_layout = kv_cache.KV_LAYOUT_TRANSPOSED
export_config.mask_as_input = True

# Configs specific for text generation models.
litertlm_config = {
    "tokenizer_model_path": os.path.join(checkpoint_dir, 'tokenizer.model'),
    "start_token_id": 2,  # "<bos>"
    "stop_token_ids": [1, 106],  # ["<eos>", "<end_of_turn>"]
    "prompt_prefix": "<start_of_turn>user\n",
    "prompt_suffix": "<end_of_turn>\n<start_of_turn>model\n",
    "model_prompt_prefix": "<start_of_turn>model\n",
    "model_prompt_suffix": "<end_of_turn>\n",
    "user_prompt_prefix": "<start_of_turn>user\n",
    "user_prompt_suffix": "<end_of_turn>\n",
    "output_format": "litertlm",
}

# Convert to LiteRT or LiteRT-LM Format
converter.convert_to_litert(
    pytorch_model,
    output_path=output_dir,
    output_name_prefix="gemma",
    prefill_seq_len=2048,
    kv_cache_max_len=4096,
    quantize="dynamic_int8",
    export_config=export_config,
    **litertlm_config
)
